# Init & Login

In [5]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

spot = spotipy.Spotify( auth = token )
clear_output( wait = False )

# Playlist Functions

In [6]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [7]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['cringe'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 0-100 --> 206:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 2, 0-100 --> 61:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 300-344 --> 146:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None

Phase 2: Chunks of 10

Iteration 1, 100-110 --> 220:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 2, 140-150 --> 2:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 3, 60-70 --> 311:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
ERROR, 340-344 --> 100:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 5, 150-160 --> 139:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 6, 60-70 --> 170:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 7, 0-10 --> 265:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 8, 150-160 --> 267:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 9, 190-200 --> 69:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 10, 60-70 --> 16:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 11, 130-140 --> 52:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 12, 50-60 --> 325:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ

HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 33, 220-230 --> 279:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
ERROR, 340-344 --> 313:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None

Phase 3: Chunks of 5, 1/2

Iteration 1, 310-315 --> 338:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 2, 335-340 --> 223:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 3, 330-335 --> 258:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 4, 140-145 --> 125:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 5, 30-35 --> 325:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 6, 250-255 --> 214:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 7, 55-60 --> 190:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 8, 70-75 --> 68:
	{'snapshot_id': 'AAAErPaS31Pi0UA3ffZVIVMUoTRUOMzQ'}
Iteration 9, 220-225 --> 96:
	{'snapshot_id':